## Configurações Iniciais

In [1]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn
import seaborn as sns
import numpy as np
import torchvision
import random
import torch

import os

def set_seed(seed=1234):

    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
!nvidia-smi

## O que é o PyTorch?
- biblioteca de aprendizado profundo

In [ ]:
tensor = torch.tensor(1)
print(tensor)

In [ ]:
tensor = torch.tensor(55)
print(tensor)

In [ ]:
tensor = torch.tensor([1,2,3])
print(tensor)
print(tensor.shape)

In [ ]:
tensor = torch.tensor([[1,2,3], [4,5,6]])
print(tensor)
print(tensor.shape)

In [ ]:
matx1 = torch.randn([3,3])
matx2 = torch.randn([3,3])
print(matx1)
print()
print(matx2)

In [2]:
matx1 + matx2

NameError: name 'matx1' is not defined

In [3]:
matx1 * matx2

NameError: name 'matx1' is not defined

In [4]:
torch.mm(matx1, matx2)

NameError: name 'torch' is not defined

In [ ]:
x = torch.tensor(3.0, requires_grad=True)
y = x**2
y.backward()
print(x.grad)

In [ ]:
x = torch.tensor(3.0, requires_grad=True)
y = x+2
y.backward()
print(x.grad)

In [ ]:
# matrix multiplication of a big matrix and compare time between gpu and cpu without timeit
x = torch.randn([1000,1000])
y = torch.randn([1000,1000])
%timeit torch.mm(x,y)

x = x.to('cuda')
y = y.to('cuda')
%timeit torch.mm(x,y)

## Preparando o Dataset
- dataset de gatos e cachorros que o professor usou
- devemos substituir por outro dataset

In [ ]:
# download dataset
! pip install gdown
! gdown --id 17zbLsM4n1QQJutNncsAaSRFj4uJVELak
!unzip /content/dataset.zip

In [ ]:
img_size = (256,256)
transformations = transforms.Compose([transforms.Resize(img_size), transforms.ToTensor()])

train = datasets.ImageFolder('./train',transform=transformations)
test= datasets.ImageFolder('./test', transform=transformations)
trainloader = DataLoader(train, batch_size=64, shuffle=True)
testloader = DataLoader(test, batch_size=64, shuffle=False)

In [ ]:
def imshow(img):
    plt.figure(figsize=(20,8))
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
images = next(dataiter)
imshow(torchvision.utils.make_grid(images[0]))

## Criando uma MLP

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            # input = (256,256,3)
            nn.Flatten(), # (196608)
            nn.Linear(256*256*3, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64,2)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
def validation(model, loader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss +=loss
    return val_loss/len(loader)

def train(model, trainloader, testloader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for data in tqdm(trainloader):
            images, labels = data
            model.zero_grad()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        val_loss = validation(model, testloader, criterion)
        print(f'Epoch: {epoch+1} | Loss: {running_loss/len(trainloader)} | Val Loss: {val_loss}')

def accuracy(model, loader):
    model.eval()
    corrected = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs, 1)
            total += labels.size(0)
            corrected += (predicted == labels).sum().item()
    return corrected * 100 // total

def confusion_matrix(model, loader):
    model.eval()
    confusion_matrix = np.zeros((2,2))
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs, 1)
            for i in range(labels.size(0)):
                confusion_matrix[labels[i].item()][predicted[i].item()] += 1
    ax = sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g', xticklabels=['Gato', 'Cachorro'], yticklabels=['Gato', 'Cachorro'])
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Label')
    return ax

In [ ]:
mlp = MLP().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
epochs = 3

In [ ]:
train(mlp, trainloader, testloader, optimizer, criterion, epochs)

In [ ]:
print(f'A rede atinge: {accuracy(mlp, testloader)}% de acurácia')
conf_mat = confusion_matrix(mlp, testloader)

In [ ]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
            #input = (256,256,3)
            nn.Conv2d(3,16, kernel_size=3, stride=1, padding=1), # (256,256,16)
            nn.ReLU(inplace=True), # (256,256,16)
            nn.MaxPool2d(kernel_size=2,stride=2), # (128,128,16)
            nn.Conv2d(16,8, kernel_size=3, stride=1, padding=1), # (128,128,8)
            nn.ReLU(inplace=True), # (128,128,8)
            nn.MaxPool2d(kernel_size=2,stride=2), # (64,64,8)
            nn.Flatten(), # (1024)
            nn.Linear(64*64*8,256), # (256)
            nn.ReLU(inplace=True),  # (256)
            nn.Linear(256,2) # (2)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
cnn = ConvolutionalNeuralNetwork().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(cnn.parameters(), lr=0.001)
epochs = 3

In [ ]:
print(f'A rede atinge: {accuracy(cnn, testloader)}% de acurácia')
conf_mat = confusion_matrix(cnn, testloader)

## Trasfer Learning
- arquitetura para reconhecimento de imagens e classificação

In [ ]:
resnet = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.DEFAULT)

In [ ]:
resnet

In [ ]:
# substituímos a última camada da rede para resolver o nosso problema de classificação
resnet.fc = nn.Linear(2048,2)

In [ ]:
resnet

In [ ]:
# "congelamos os pesos das camadas, exceto a última"
for name, params in resnet.named_parameters():
    if name not in ('fc.weight', 'fc.bias'):
        params.requires_grad = False
# for name, params in resnet.named_parameters():
#     print(name, params.requires_grad)

In [ ]:
resnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
epochs = 3

In [ ]:
train(resnet, trainloader, testloader, optimizer, criterion, epochs)

In [ ]:
print(f'A rede atinge: {accuracy(resnet, testloader)}% de acurácia')
conf_mat = confusion_matrix(resnet, testloader)

## Comparando os Modelos

In [ ]:
def predict(model, image):
    prediction = model(torch.unsqueeze(image, 0).to(device))
    result = torch.argmax(prediction)
    return 'Cat' if result == 0 else 'Dog'

In [ ]:
image_index = 20
image = test[image_index][0]
label = 'Dog' if test[image_index][1] else 'Cat'
imshow(image)
print(label)

In [ ]:
print(f'Real Label: {label}')
print(f'MLP Prediction: {predict(mlp, image)}')
print(f'CNN Prediction: {predict(cnn, image)}')
print(f'Resnet Prediction: {predict(resnet, image)}')

In [ ]:
from PIL import Image
import torchvision.transforms.functional as TF

x = Image.open('caramelo.jpg').convert('RGB')
x = transformations(x)

In [ ]:
print(f'MLP Prediction: {predict(mlp, x)}')
print(f'CNN Prediction: {predict(cnn, x)}')
print(f'Resnet Prediction: {predict(resnet, x)}')